## Challenge

In this lab, the objective is to identify the customers who were active in both May and June, and how did their activity differ between months. To achieve this, follow these steps:

1. Establish a connection between Python and the Sakila database.

In [1]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
import getpass  # To get the password without showing the input


In [2]:
password = getpass.getpass()

In [3]:
# Configuración de la base de datos y la conexión
bd = "sakila"
connection_string = 'mysql+pymysql://root:' + password + '@localhost/' + bd
engine = create_engine(connection_string)

2. Write a Python function called `rentals_month` that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

	- `engine`: an object representing the database connection engine to be used to establish a connection to the Sakila database.
	- `month`: an integer representing the month for which rental data is to be retrieved.
	- `year`: an integer representing the year for which rental data is to be retrieved.

	The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.

In [4]:
def rentals_month(engine, month, year):
    query = f"""
        SELECT rental_id, rental_date, customer_id
        FROM rental
        WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    """
    # Execute the SQL query and return the result as a DataFrame
    with engine.connect() as connection:
        df = pd.read_sql(query, connection)
    return df

In [5]:
df = rentals_month(engine, 5, 2005)

3. Develop a Python function called `rental_count_month` that takes the DataFrame provided by `rentals_month` as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year. 

	The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".


	*Hint: Consider making use of pandas [groupby()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)*

In [6]:
def rental_count_month(rentals_df, month, year):
    # Group by 'customer_id' and count the rentals
    rental_count = rentals_df.groupby('customer_id').size().reset_index(name=f'rentals_{month:02}_{year}')
    return rental_count

In [7]:
rental_count_month(df, 5, 2005)

,customer_id,rentals_05_2005
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3
...,...,...
515,594,4
516,595,1
517,596,6
518,597,2


4. Create a Python function called `compare_rentals` that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. 
The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

In [8]:
def compare_rentals(df1, df2):
    df = pd.merge(df1, df2, on='customer_id', how='outer')
    df['difference'] = df[df.columns[1]] - df[df.columns[2]]
    return df


In [9]:
df1 = rentals_month(engine, 5, 2005)
df2 = rentals_month(engine, 6, 2005)

In [10]:
df1 = rental_count_month(df1, 5, 2005)
df2 = rental_count_month(df2, 6, 2005)

In [11]:
compare_rentals(df1, df2)

,customer_id,rentals_05_2005,rentals_06_2005,difference
0,1,2.0,7.0,-5.0
1,2,1.0,1.0,0.0
2,3,2.0,4.0,-2.0
3,4,NaN,6.0,NaN
4,5,3.0,5.0,-2.0
...,...,...,...,...
593,595,1.0,2.0,-1.0
594,596,6.0,2.0,4.0
595,597,2.0,3.0,-1.0
596,598,NaN,1.0,NaN
